In [0]:
## ALGORITMO GENETICO COMPACTO POP COMPLETO e SIMPLES COMPARACAO EXECUÇÂO FTRAP5
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

###################          SGA        ###################

def generate_pop_sga(size,population_size):
    sga_vector_pop = []

    for ps in range(population_size):
      sga_value = ''
      for s in range(size):
        sga_value += '1' if random.random() > 0.5  else '0'
      sga_vector_pop.append( {'val':sga_value,'fit':0} )

    return sga_vector_pop

def crossover_sga(size,population,mutation):
    son = population[0].copy()
    son2 = population[1].copy()

    s = son['val']
    s2 = son2['val']
    middle = random.randrange(1,size-1)
    
    son['fit'] = 0
    son2['fit'] = 0
    son['val'] = s[:middle] + s2[middle:]
    son2['val'] = s2[:middle] + s[middle:]
  
    # mutation
    child = son['val']
    child2 = son2['val']
    c1 = ""
    for c in child:
      mut = random.random()
      if(mut < mutation):
        c1+='1' if c == '0' else '0'
      else:
        c1+=c
    son['val'] = c1

    c2 = ""
    for s in child2:
      mut = random.random()
      if(mut < mutation):
        c2+='1' if s == '0' else '0'
      else:
        c2+=s
    son2['val'] = c2
    return son,son2

def run_sga(generations, size, population_size):
    # criação da população aleatoria
    population = generate_pop_sga(size,population_size)
    population = fitness_function_sga(population)

    mutation = 0.01

    # percorre as gerações
    for i in range(generations):

      mid = int(population_size/2) if int(population_size/2) % 2 == 0 else int(population_size/2)-1
      new_population = []
      
      for j in range(0,mid,2):
        new_population += crossover_sga(size,population[j:j+2],mutation)
        #print(j,new_population)

      full_pop = population + new_population
      full_pop = fitness_function_sga(full_pop)

      population = full_pop[:population_size]
      
      #print ("generation: {} and population {}".format(i + 1,population))
    p = population[0]
    return p['fit']

def sort_fit(e):
  return e['fit']

# Define fitness function (ftrap5)
def ftrap5(individual):
  u = 0
  for a in individual:
    u += int(a)
  if u < 5:
    return 4 - u
  return 5

# funcao de fitness
def fitness_function_sga(population):
  vector_fit = []

  for p in population:
    ft = 0
    individuo = p['val']
    for i in [individuo[:5], individuo[5:10], individuo[10:15]]:
      ft+= ftrap5(i)
    p['fit']= ft
    vector_fit.append(p)

  vector_fit.sort(reverse = True,key=sort_fit)
  
  return vector_fit
###################          CGA        ###################

class Solution(object):
    def __init__(self, value):
        self.value = value
        self.fitness = 0

    def calculate_fitness(self):
        self.fitness = fitness_function(self.value)

def generate_candidate(vector,population_size):
    pop=[]
    for n in range(population_size):
      value = ''
      for q in vector:
        value += "1" if random.random() < q else "0"
      pop.append(value)
    
    #print(pop)
    return Solution(pop)

def generate_vector(size):
    return [0.5] * size

def compete(a, b, a1, b1):
    if a > b:
        return a1, b1
    else:
        return b1, a1

def competition(population):
    for n in range(1,len(population.value)):
      winner,loser = compete(population.fitness[0],population.fitness[n],population.value[0],population.value[n])
    
    return winner, loser

def update_vector(vector, winner, loser, population_size):
    for i in range(len(vector)):
        if winner[i] != loser[i]:
            if winner[i] == '1':
                vector[i] += 1.0 / float(population_size)
            else:
                vector[i] -= 1.0 / float(population_size)

def run_cga(generations, size, population_size):
    # probabilidade do vetor
    vector = generate_vector(size)
    #print(vector)
    best = None
    for i in range(generations):
      population = generate_candidate(vector, population_size)
      population.calculate_fitness()
      winner, loser = competition(population)
      
      # verifica qual é o melhor
      if best:
          if int(winner,2) > int(best,2):
             best = winner
      else:
          best = winner
      
      update_vector(vector,winner,loser,population_size)
      
      #print ("generation: {} best value: {} best fitness: {}".format(i + 1, best, int(best,2) ) )
    p = 0
    for i in [best[:5], best[5:10], best[10:15]]:
      p+= ftrap5(i)
    return p

def fitness_function(value):
  value.sort(reverse=True)

  fit =[]

  for n in value:
    ft=0
    for i in [n[:5], n[5:10], n[10:15]]:
      ft+= ftrap5(i)
    fit.append(ft)
  
  return fit

tests = 10
ind = [10,20,40,80,160]

def plotgraphics(progress_cga,progress_sga,pop_size):
  cga_accuracy = 0.0
  for c in progress_cga:
    cga_accuracy += c / 15
  cga_accuracy /= tests

  sga_accuracy = 0.0
  for s in progress_sga:
    sga_accuracy += s / 15
  sga_accuracy /= tests

  # Plot
  fig = plt.figure()
  fig.suptitle('População de ' + str(pop_size) + ' individuos',
     fontsize=14, fontweight='bold')
  line_cga, = plt.plot(progress_cga)
  line_sga, = plt.plot(progress_sga)
  #plt.ylim(0, 100)
  plt.legend([line_sga, line_cga],
    ['sGA ({:.2f}%)'.format(sga_accuracy*100),
     'cGA ({:.2f}%)'.format(cga_accuracy*100)])
  plt.xlabel('Execução')
  plt.ylabel('Melhor fitness')
  plt.show()
  
progress_cga = []
progress_sga = []

for i in ind:
  cgafitness = []
  sgafitness = []
  for _ in range(tests):
    cgafitness.append( run_cga(100, 15, i) )
    sgafitness.append( run_sga(100, 15, i) )
  progress_cga.append(cgafitness)
  progress_sga.append(sgafitness)

for i in range( len(ind) ):
  plotgraphics(progress_cga[i],progress_sga[i],ind[i])